# Unit 4: Handling Missing Values - Practical Exercises

## Table of Contents
1. [Introduction](#introduction)
2. [Setup](#setup)
3. [Detecting Missing Values](#detecting-missing-values)
4. [Understanding Missingness Patterns](#understanding-patterns)
5. [Deletion Methods](#deletion-methods)
6. [Simple Imputation Techniques](#simple-imputation)
7. [Advanced Imputation Methods](#advanced-imputation)
8. [Creating Missing Indicators](#missing-indicators)
9. [Validating Imputation Quality](#validation)
10. [Complete Workflow Example](#complete-workflow)

---

## 1. Introduction <a id='introduction'></a>

In this practical notebook, you'll learn to handle missing data systematically using various techniques. You'll work with real datasets and apply deletion, simple imputation, and advanced imputation methods.

**Learning Objectives:**
- Detect and visualize missing data patterns
- Understand MCAR, MAR, and MNAR mechanisms
- Apply deletion methods appropriately
- Implement simple and advanced imputation techniques
- Validate imputation quality
- Create missing value indicators for modeling

## 2. Setup <a id='setup'></a>

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import ks_2samp

# Settings
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)
np.random.seed(42)

print("✅ Setup complete!")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")

## 3. Detecting Missing Values <a id='detecting-missing-values'></a>

Let's start by loading a dataset and detecting missing values.

In [ ]:
# Load Titanic dataset (has missing values)
titanic = sns.load_dataset('titanic')

print("TITANIC DATASET - MISSING VALUES DETECTION")
print("="*80)
print(f"Dataset shape: {titanic.shape}")
print(f"Total cells: {titanic.shape[0] * titanic.shape[1]}")
print(f"Total missing cells: {titanic.isnull().sum().sum()}")
print(f"Overall missing percentage: {(titanic.isnull().sum().sum() / (titanic.shape[0] * titanic.shape[1]) * 100):.2f}%")

In [ ]:
# Count missing values per column
print("\nMISSING VALUES PER COLUMN")
print("="*80)

missing_count = titanic.isnull().sum()
missing_pct = (missing_count / len(titanic) * 100).round(2)

missing_summary = pd.DataFrame({
    'Missing Count': missing_count,
    'Missing %': missing_pct,
    'Present Count': len(titanic) - missing_count
})

# Filter and sort by missing percentage
missing_summary = missing_summary[missing_summary['Missing Count'] > 0].sort_values('Missing %', ascending=False)

# Add severity indicator
def categorize_severity(pct):
    if pct < 5: return '🟢 Low'
    elif pct < 20: return '🟡 Medium'
    else: return '🔴 High'

missing_summary['Severity'] = missing_summary['Missing %'].apply(categorize_severity)

print(missing_summary)

In [ ]:
# Visualize missing data patterns
print("\nMISSING DATA VISUALIZATIONS")
print("="*80)

# 1. Missing data matrix
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Matrix visualization
msno.matrix(titanic, ax=axes[0, 0], fontsize=10)
axes[0, 0].set_title('Missing Data Matrix', fontsize=12, fontweight='bold')

# Bar chart
missing_cols = titanic.columns[titanic.isnull().any()].tolist()
missing_data = titanic[missing_cols].isnull().sum().sort_values(ascending=True)
axes[0, 1].barh(missing_data.index, missing_data.values, color='coral')
axes[0, 1].set_xlabel('Number of Missing Values')
axes[0, 1].set_title('Missing Values by Column', fontweight='bold')
axes[0, 1].grid(axis='x', alpha=0.3)

# Heatmap of missing values
axes[1, 0].set_title('Missing Data Heatmap (sample)', fontweight='bold')
sns.heatmap(titanic.head(50).isnull(), cbar=False, yticklabels=False, 
            cmap='viridis', ax=axes[1, 0])

# Missing values per row distribution
missing_per_row = titanic.isnull().sum(axis=1)
axes[1, 1].hist(missing_per_row, bins=range(0, missing_per_row.max()+2), 
                edgecolor='black', alpha=0.7, color='steelblue')
axes[1, 1].set_xlabel('Number of Missing Values per Row')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Distribution of Missing Values per Row', fontweight='bold')
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nRows with no missing values: {(missing_per_row == 0).sum()}")
print(f"Rows with at least 1 missing value: {(missing_per_row > 0).sum()}")
print(f"Maximum missing values in a row: {missing_per_row.max()}")

## 4. Understanding Missingness Patterns <a id='understanding-patterns'></a>

Let's analyze whether missingness is related to other variables (checking for MAR vs MCAR).

In [ ]:
# Check if age missingness is related to other variables
print("ANALYZING AGE MISSINGNESS PATTERN")
print("="*80)

# Compare characteristics of observations with vs without missing age
age_missing = titanic[titanic['age'].isna()]
age_present = titanic[titanic['age'].notna()]

print(f"\nObservations with missing age: {len(age_missing)}")
print(f"Observations with present age: {len(age_present)}")

print("\n" + "="*80)
print("COMPARISON: Missing Age vs Present Age")
print("="*80)

comparison_vars = ['pclass', 'fare', 'survived']
for var in comparison_vars:
    if var in titanic.columns:
        missing_mean = age_missing[var].mean()
        present_mean = age_present[var].mean()
        diff = abs(missing_mean - present_mean)
        
        print(f"\n{var.upper()}:")
        print(f"  Missing age group mean: {missing_mean:.2f}")
        print(f"  Present age group mean: {present_mean:.2f}")
        print(f"  Difference: {diff:.2f}")
        
        if diff < 0.1:
            print(f"  → Consistent with MCAR")
        else:
            print(f"  → Suggests MAR (missingness related to {var})")

In [ ]:
# Correlation of missingness between variables
print("\nCORRELATION OF MISSINGNESS")
print("="*80)

# Create missingness indicators
missing_indicators = titanic[missing_cols].isnull().astype(int)
missing_indicators.columns = [f'{col}_missing' for col in missing_cols]

# Calculate correlation
missing_corr = missing_indicators.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(missing_corr, annot=True, cmap='coolwarm', center=0, 
            fmt='.2f', square=True, linewidths=1)
plt.title('Correlation Between Missing Value Patterns', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("- High positive correlation: Variables tend to be missing together")
print("- Low correlation: Missing patterns are independent")

## 5. Deletion Methods <a id='deletion-methods'></a>

Let's explore different deletion approaches and their impact.

In [ ]:
# Listwise deletion (complete case analysis)
print("LISTWISE DELETION")
print("="*80)

titanic_complete = titanic.dropna()

print(f"Original dataset: {titanic.shape[0]} rows")
print(f"After listwise deletion: {titanic_complete.shape[0]} rows")
print(f"Data loss: {(1 - len(titanic_complete)/len(titanic)) * 100:.1f}%")

# Check if deletion introduced bias
print("\n" + "="*80)
print("CHECKING FOR BIAS")
print("="*80)

for col in ['age', 'fare']:
    if col in titanic.columns:
        original_mean = titanic[col].mean()
        complete_mean = titanic_complete[col].mean()
        
        print(f"\n{col.upper()}:")
        print(f"  Original mean: {original_mean:.2f}")
        print(f"  Complete cases mean: {complete_mean:.2f}")
        print(f"  Difference: {abs(original_mean - complete_mean):.2f}")

In [ ]:
# Column deletion
print("\nCOLUMN DELETION")
print("="*80)

threshold = 0.5  # Drop columns with >50% missing
missing_pct = titanic.isnull().mean()
cols_to_drop = missing_pct[missing_pct > threshold].index.tolist()

print(f"Columns with >{threshold*100}% missing:")
print(cols_to_drop)

if cols_to_drop:
    titanic_reduced = titanic.drop(columns=cols_to_drop)
    print(f"\nOriginal columns: {titanic.shape[1]}")
    print(f"After column deletion: {titanic_reduced.shape[1]}")
    print(f"Columns dropped: {len(cols_to_drop)}")
else:
    print("No columns exceed threshold")
    titanic_reduced = titanic.copy()

In [ ]:
# Pairwise deletion example (correlation)
print("\nPAIRWISE DELETION (Correlation Analysis)")
print("="*80)

# Select numerical columns
numerical_cols = ['age', 'fare', 'pclass', 'survived']
subset = titanic[numerical_cols]

print("Pairwise deletion uses all available data for each pair:")
print("\nSample sizes for each correlation:")

for i, col1 in enumerate(numerical_cols):
    for col2 in numerical_cols[i+1:]:
        valid_pairs = subset[[col1, col2]].dropna()
        print(f"  {col1} vs {col2}: {len(valid_pairs)} pairs")

# Compute correlation with pairwise deletion
corr_pairwise = subset.corr()

plt.figure(figsize=(8, 6))
sns.heatmap(corr_pairwise, annot=True, cmap='coolwarm', center=0, 
            fmt='.2f', square=True, linewidths=1)
plt.title('Correlation Matrix (Pairwise Deletion)', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

## 6. Simple Imputation Techniques <a id='simple-imputation'></a>

Now let's implement various simple imputation methods.

In [ ]:
# Mean/Median/Mode imputation
print("SIMPLE IMPUTATION METHODS")
print("="*80)

# Create copies for different methods
df_mean = titanic.copy()
df_median = titanic.copy()
df_mode = titanic.copy()

# Age: Mean imputation
age_mean = df_mean['age'].mean()
df_mean['age'].fillna(age_mean, inplace=True)

# Age: Median imputation
age_median = df_median['age'].median()
df_median['age'].fillna(age_median, inplace=True)

# Embarked: Mode imputation
embarked_mode = df_mode['embark_town'].mode()[0]
df_mode['embark_town'].fillna(embarked_mode, inplace=True)

print(f"Age - Original mean: {titanic['age'].mean():.2f}")
print(f"Age - After mean imputation: {df_mean['age'].mean():.2f}")
print(f"Age - After median imputation: {df_median['age'].mean():.2f}")

print(f"\nAge - Original std: {titanic['age'].std():.2f}")
print(f"Age - After mean imputation: {df_mean['age'].std():.2f}")
print(f"Age - After median imputation: {df_median['age'].std():.2f}")
print("\n⚠️ Notice: Variance decreased after imputation!")

In [ ]:
# Visualize impact of imputation
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Original distribution
axes[0].hist(titanic['age'].dropna(), bins=30, alpha=0.7, 
             color='steelblue', edgecolor='black')
axes[0].set_xlabel('Age')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Original Distribution (Missing Removed)', fontweight='bold')
axes[0].axvline(titanic['age'].mean(), color='red', linestyle='--', 
                linewidth=2, label=f'Mean: {titanic["age"].mean():.1f}')
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Mean imputation
axes[1].hist(df_mean['age'], bins=30, alpha=0.7, 
             color='orange', edgecolor='black')
axes[1].set_xlabel('Age')
axes[1].set_ylabel('Frequency')
axes[1].set_title('After Mean Imputation', fontweight='bold')
axes[1].axvline(df_mean['age'].mean(), color='red', linestyle='--', 
                linewidth=2, label=f'Mean: {df_mean["age"].mean():.1f}')
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

# Median imputation
axes[2].hist(df_median['age'], bins=30, alpha=0.7, 
             color='green', edgecolor='black')
axes[2].set_xlabel('Age')
axes[2].set_ylabel('Frequency')
axes[2].set_title('After Median Imputation', fontweight='bold')
axes[2].axvline(df_median['age'].mean(), color='red', linestyle='--', 
                linewidth=2, label=f'Mean: {df_median["age"].mean():.1f}')
axes[2].legend()
axes[2].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Constant value imputation
print("\nCONSTANT VALUE IMPUTATION")
print("="*80)

df_constant = titanic.copy()

# Fill embarked with 'Unknown'
df_constant['embark_town'].fillna('Unknown', inplace=True)

print("Embarked - Value counts after constant imputation:")
print(df_constant['embark_town'].value_counts())

## 7. Advanced Imputation Methods <a id='advanced-imputation'></a>

Let's implement KNN and MICE imputation.

In [ ]:
# KNN Imputation
print("KNN IMPUTATION")
print("="*80)

# Select numerical columns for imputation
numerical_features = ['age', 'fare', 'pclass', 'sibsp', 'parch']
df_knn = titanic[numerical_features].copy()

print(f"Missing values before KNN imputation:")
print(df_knn.isnull().sum())

# Apply KNN imputation
knn_imputer = KNNImputer(n_neighbors=5, weights='uniform')
df_knn_imputed = pd.DataFrame(
    knn_imputer.fit_transform(df_knn),
    columns=df_knn.columns,
    index=df_knn.index
)

print(f"\nMissing values after KNN imputation:")
print(df_knn_imputed.isnull().sum())

# Compare statistics
print("\n" + "="*80)
print("COMPARISON: Original vs KNN Imputation")
print("="*80)

for col in ['age', 'fare']:
    print(f"\n{col.upper()}:")
    print(f"  Original mean: {df_knn[col].mean():.2f}, std: {df_knn[col].std():.2f}")
    print(f"  After KNN: {df_knn_imputed[col].mean():.2f}, std: {df_knn_imputed[col].std():.2f}")
    print(f"  Variance preservation: {(df_knn_imputed[col].std() / df_knn[col].std() * 100):.1f}%")

In [ ]:
# Visualize KNN imputation results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Age comparison
axes[0].hist(titanic['age'].dropna(), bins=30, alpha=0.5, 
             label='Original', color='steelblue', edgecolor='black')
axes[0].hist(df_knn_imputed['age'], bins=30, alpha=0.5, 
             label='KNN Imputed', color='orange', edgecolor='black')
axes[0].set_xlabel('Age')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Age Distribution: Original vs KNN Imputation', fontweight='bold')
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Fare comparison
axes[1].hist(titanic['fare'].dropna(), bins=30, alpha=0.5, 
             label='Original', color='steelblue', edgecolor='black')
axes[1].hist(df_knn_imputed['fare'], bins=30, alpha=0.5, 
             label='KNN Imputed', color='orange', edgecolor='black')
axes[1].set_xlabel('Fare')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Fare Distribution: Original vs KNN Imputation', fontweight='bold')
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# MICE (Iterative Imputation)
print("\nMICE (MULTIVARIATE IMPUTATION BY CHAINED EQUATIONS)")
print("="*80)

df_mice = titanic[numerical_features].copy()

# Apply MICE
mice_imputer = IterativeImputer(max_iter=10, random_state=42, verbose=0)
df_mice_imputed = pd.DataFrame(
    mice_imputer.fit_transform(df_mice),
    columns=df_mice.columns,
    index=df_mice.index
)

print(f"Missing values after MICE imputation:")
print(df_mice_imputed.isnull().sum())

# Compare statistics
print("\n" + "="*80)
print("COMPARISON: Original vs MICE Imputation")
print("="*80)

for col in ['age', 'fare']:
    print(f"\n{col.upper()}:")
    print(f"  Original mean: {df_mice[col].mean():.2f}, std: {df_mice[col].std():.2f}")
    print(f"  After MICE: {df_mice_imputed[col].mean():.2f}, std: {df_mice_imputed[col].std():.2f}")

In [ ]:
# Compare all imputation methods
print("\nCOMPARISON OF ALL IMPUTATION METHODS")
print("="*80)

comparison_data = {
    'Method': ['Original', 'Mean', 'Median', 'KNN', 'MICE'],
    'Age Mean': [
        titanic['age'].mean(),
        df_mean['age'].mean(),
        df_median['age'].mean(),
        df_knn_imputed['age'].mean(),
        df_mice_imputed['age'].mean()
    ],
    'Age Std': [
        titanic['age'].std(),
        df_mean['age'].std(),
        df_median['age'].std(),
        df_knn_imputed['age'].std(),
        df_mice_imputed['age'].std()
    ]
}

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.round(2)

print(comparison_df.to_string(index=False))

print("\n" + "="*80)
print("KEY OBSERVATIONS:")
print("="*80)
print("✅ KNN and MICE better preserve variance compared to mean/median")
print("✅ Mean imputation maintains original mean but reduces std")
print("✅ Advanced methods capture relationships between variables")

## 8. Creating Missing Indicators <a id='missing-indicators'></a>

Missing indicators help machine learning models learn from the pattern of missingness.

In [ ]:
# Create missing indicators
print("CREATING MISSING INDICATORS")
print("="*80)

df_with_indicators = titanic.copy()

# Create indicators for columns with missing values
missing_cols = titanic.columns[titanic.isnull().any()].tolist()

for col in missing_cols:
    indicator_name = f'{col}_was_missing'
    df_with_indicators[indicator_name] = titanic[col].isnull().astype(int)
    
    print(f"{col}: {df_with_indicators[indicator_name].sum()} missing values marked")

print(f"\nOriginal columns: {titanic.shape[1]}")
print(f"After adding indicators: {df_with_indicators.shape[1]}")
print(f"New indicator columns: {len(missing_cols)}")

In [ ]:
# Now impute the original columns
print("\nIMPUTING AFTER CREATING INDICATORS")
print("="*80)

# Use median for numerical, mode for categorical
for col in missing_cols:
    if df_with_indicators[col].dtype in [np.float64, np.int64]:
        df_with_indicators[col].fillna(df_with_indicators[col].median(), inplace=True)
    else:
        mode_val = df_with_indicators[col].mode()
        if len(mode_val) > 0:
            df_with_indicators[col].fillna(mode_val[0], inplace=True)

print(f"Missing values after imputation: {df_with_indicators.isnull().sum().sum()}")
print("\n✅ Dataset now has:")
print("  - Original features (imputed)")
print("  - Missing indicators (for modeling)")

## 9. Validating Imputation Quality <a id='validation'></a>

Let's validate imputation quality by creating artificial missing data.

In [ ]:
# Validation by artificial missing data
print("VALIDATING IMPUTATION QUALITY")
print("="*80)

# Create test dataset with artificial missing values
test_data = titanic[['age', 'fare', 'pclass', 'sibsp', 'parch']].dropna().copy()

print(f"Complete cases for testing: {len(test_data)}")

# Randomly remove 20% of age values
np.random.seed(42)
mask = np.random.rand(len(test_data)) < 0.2
true_values = test_data.loc[mask, 'age'].copy()
test_data.loc[mask, 'age'] = np.nan

print(f"Artificial missing values created: {mask.sum()}")

In [ ]:
# Test different imputation methods
from sklearn.metrics import mean_squared_error, mean_absolute_error

results = {}

# 1. Mean imputation
test_mean = test_data.copy()
test_mean['age'].fillna(test_mean['age'].mean(), inplace=True)
mean_imputed = test_mean.loc[mask, 'age']

results['Mean'] = {
    'RMSE': np.sqrt(mean_squared_error(true_values, mean_imputed)),
    'MAE': mean_absolute_error(true_values, mean_imputed)
}

# 2. Median imputation
test_median = test_data.copy()
test_median['age'].fillna(test_median['age'].median(), inplace=True)
median_imputed = test_median.loc[mask, 'age']

results['Median'] = {
    'RMSE': np.sqrt(mean_squared_error(true_values, median_imputed)),
    'MAE': mean_absolute_error(true_values, median_imputed)
}

# 3. KNN imputation
test_knn = test_data.copy()
knn_imp = KNNImputer(n_neighbors=5)
test_knn_values = knn_imp.fit_transform(test_knn)
test_knn = pd.DataFrame(test_knn_values, columns=test_knn.columns, index=test_knn.index)
knn_imputed = test_knn.loc[mask, 'age']

results['KNN'] = {
    'RMSE': np.sqrt(mean_squared_error(true_values, knn_imputed)),
    'MAE': mean_absolute_error(true_values, knn_imputed)
}

# 4. MICE imputation
test_mice = test_data.copy()
mice_imp = IterativeImputer(max_iter=10, random_state=42)
test_mice_values = mice_imp.fit_transform(test_mice)
test_mice = pd.DataFrame(test_mice_values, columns=test_mice.columns, index=test_mice.index)
mice_imputed = test_mice.loc[mask, 'age']

results['MICE'] = {
    'RMSE': np.sqrt(mean_squared_error(true_values, mice_imputed)),
    'MAE': mean_absolute_error(true_values, mice_imputed)
}

# Display results
print("\n" + "="*80)
print("IMPUTATION QUALITY COMPARISON")
print("="*80)

results_df = pd.DataFrame(results).T
results_df = results_df.round(2)
print(results_df)

print("\n" + "="*80)
print("INTERPRETATION:")
print("="*80)
print("✅ Lower RMSE and MAE indicate better imputation quality")
print(f"✅ Best method: {results_df['RMSE'].idxmin()} (lowest RMSE)")

In [ ]:
# Visualize prediction accuracy
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

methods = ['Mean', 'Median', 'KNN', 'MICE']
imputed_values = [mean_imputed, median_imputed, knn_imputed, mice_imputed]

for idx, (method, imputed) in enumerate(zip(methods, imputed_values)):
    row = idx // 2
    col = idx % 2
    
    axes[row, col].scatter(true_values, imputed, alpha=0.6, s=50)
    axes[row, col].plot([true_values.min(), true_values.max()], 
                         [true_values.min(), true_values.max()], 
                         'r--', linewidth=2, label='Perfect Prediction')
    axes[row, col].set_xlabel('True Age')
    axes[row, col].set_ylabel('Imputed Age')
    axes[row, col].set_title(f'{method} Imputation\nRMSE: {results[method]["RMSE"]:.2f}', 
                             fontweight='bold')
    axes[row, col].legend()
    axes[row, col].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Complete Workflow Example <a id='complete-workflow'></a>

Let's put it all together in a complete workflow.

In [ ]:
def handle_missing_data(df, strategy='auto'):
    """
    Complete workflow for handling missing data.
    
    Parameters:
    -----------
    df : DataFrame
        Input dataframe with missing values
    strategy : str
        'auto', 'simple', 'knn', or 'mice'
    
    Returns:
    --------
    df_clean : DataFrame
        Cleaned dataframe
    report : dict
        Summary of actions taken
    """
    
    df_clean = df.copy()
    report = {
        'original_shape': df.shape,
        'missing_before': df.isnull().sum().sum(),
        'actions': []
    }
    
    print("="*80)
    print("MISSING DATA HANDLING WORKFLOW")
    print("="*80)
    
    # Step 1: Identify missing values
    missing_pct = df.isnull().mean()
    cols_with_missing = missing_pct[missing_pct > 0].index.tolist()
    
    print(f"\nStep 1: Identified {len(cols_with_missing)} columns with missing values")
    
    # Step 2: Drop columns with >70% missing
    threshold = 0.7
    cols_to_drop = missing_pct[missing_pct > threshold].index.tolist()
    
    if cols_to_drop:
        df_clean = df_clean.drop(columns=cols_to_drop)
        report['actions'].append(f"Dropped {len(cols_to_drop)} columns with >{threshold*100}% missing")
        print(f"Step 2: Dropped {len(cols_to_drop)} columns")
    else:
        print(f"Step 2: No columns exceed {threshold*100}% missing threshold")
    
    # Step 3: Create missing indicators
    remaining_missing = df_clean.columns[df_clean.isnull().any()].tolist()
    
    for col in remaining_missing:
        df_clean[f'{col}_was_missing'] = df_clean[col].isnull().astype(int)
    
    if remaining_missing:
        print(f"Step 3: Created {len(remaining_missing)} missing indicators")
        report['actions'].append(f"Created {len(remaining_missing)} missing indicators")
    
    # Step 4: Impute based on strategy
    if strategy == 'simple' or (strategy == 'auto' and df.shape[0] > 10000):
        # Simple imputation for large datasets
        for col in remaining_missing:
            if df_clean[col].dtype in [np.float64, np.int64]:
                df_clean[col].fillna(df_clean[col].median(), inplace=True)
            else:
                mode_val = df_clean[col].mode()
                if len(mode_val) > 0:
                    df_clean[col].fillna(mode_val[0], inplace=True)
        
        method = 'Simple (median/mode)'
        
    elif strategy == 'knn' or strategy == 'auto':
        # KNN imputation
        numerical_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()
        numerical_cols = [col for col in numerical_cols if not col.endswith('_was_missing')]
        
        if numerical_cols:
            imputer = KNNImputer(n_neighbors=5)
            df_clean[numerical_cols] = imputer.fit_transform(df_clean[numerical_cols])
        
        # Mode for categorical
        categorical_cols = df_clean.select_dtypes(include=['object', 'category']).columns.tolist()
        for col in categorical_cols:
            if df_clean[col].isnull().any():
                mode_val = df_clean[col].mode()
                if len(mode_val) > 0:
                    df_clean[col].fillna(mode_val[0], inplace=True)
        
        method = 'KNN (k=5) + mode'
    
    print(f"Step 4: Applied {method} imputation")
    report['actions'].append(f"Imputed using {method}")
    
    # Step 5: Validate
    report['missing_after'] = df_clean.isnull().sum().sum()
    report['final_shape'] = df_clean.shape
    
    print(f"\nStep 5: Validation")
    print(f"  Missing values before: {report['missing_before']}")
    print(f"  Missing values after: {report['missing_after']}")
    print(f"  Shape: {report['original_shape']} → {report['final_shape']}")
    
    print("\n" + "="*80)
    print("✅ WORKFLOW COMPLETE")
    print("="*80)
    
    return df_clean, report

In [ ]:
# Apply complete workflow
print("APPLYING COMPLETE WORKFLOW TO TITANIC DATASET")
print("\n")

titanic_cleaned, workflow_report = handle_missing_data(titanic, strategy='knn')

# Display final report
print("\n" + "="*80)
print("FINAL REPORT")
print("="*80)

for action in workflow_report['actions']:
    print(f"✓ {action}")

print(f"\nDataset ready for analysis!")
print(f"Shape: {workflow_report['final_shape']}")
print(f"Missing values: {workflow_report['missing_after']}")

In [ ]:
# Verify no missing values remain
print("\nFINAL VERIFICATION")
print("="*80)

missing_check = titanic_cleaned.isnull().sum()
missing_check = missing_check[missing_check > 0]

if len(missing_check) == 0:
    print("✅ SUCCESS: No missing values remain!")
else:
    print("⚠️ WARNING: Some missing values still present:")
    print(missing_check)

# Show sample of cleaned data
print("\n" + "="*80)
print("SAMPLE OF CLEANED DATA")
print("="*80)
print(titanic_cleaned.head(10))

## Summary

**What We Accomplished:**

1. ✅ Detected and visualized missing data patterns
2. ✅ Analyzed missingness mechanisms (MCAR, MAR, MNAR)
3. ✅ Applied deletion methods (listwise, pairwise, column)
4. ✅ Implemented simple imputation (mean, median, mode, constant)
5. ✅ Applied advanced imputation (KNN, MICE)
6. ✅ Created missing indicators for modeling
7. ✅ Validated imputation quality
8. ✅ Built complete automated workflow

**Key Takeaways:**

- 🎯 **Understand before imputing** - Check missingness patterns first
- 🎯 **No universal solution** - Choose method based on data and goals
- 🎯 **Preserve information** - Use missing indicators
- 🎯 **Validate quality** - Test imputation accuracy
- 🎯 **Advanced > Simple** - KNN and MICE preserve variance better
- 🎯 **Document everything** - Track all decisions made

**Next Steps:**

- Practice with different datasets
- Experiment with various imputation methods
- Complete the exercises notebook
- Move to Unit 5: Univariate Analysis